<a href="https://colab.research.google.com/github/GunmetalPoem/Visualizer-HTML/blob/main/CNN_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run Below


In [ ]:
# @title Run this to setup libraries and environment
!pip -q install streamlit > /dev/null
!pip -q install pyngrok > /dev/null
!pip install streamlit
!pip install opencv-python
!pip install streamlit-webrtc


import tensorflow as tf
import pandas as pd
import zipfile
import gdown
import cv2
import plotly.express as px
import numpy as np
from pyngrok import ngrok
import seaborn as sns
import matplotlib.pyplot as plt

def launch_website():
  print ("Click this link to try your web app:")
  if (ngrok.get_tunnels() != None):
    ngrok.kill()
  public_url = ngrok.connect()
  print (public_url)
  !streamlit run --server.port 80 app.py >/dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.4/873.4 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 100.0 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 3.4.8
    Uninstalling cryptography-3.4.8:
      Successfully uninstalled cryptography-3.4.8


Running Ngrok Authtoken

In [ ]:
!ngrok authtoken 2T8QECO37MfUyO7Uu138pRtpjlV_5rVT3ZTenXPxp6nUGbPs5

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Importing the cnn model


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

cnn_path = F"/content/gdrive/My Drive/transfer_cnn.zip"

with zipfile.ZipFile(cnn_path, 'r') as zip_ref:
    zip_ref.extractall('')

Mounted at /content/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


Make model = your imported file


In [ ]:
model = tf.keras.models.load_model('transfer_cnn')

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 46, 46, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 64)       0         
 )                                                               
                                                                 
 dropout_6 (Dropout)         (None, 23, 23, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 21, 21, 64)      

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
from streamlit_webrtc import webrtc_streamer, VideoTransformerBase, RTCConfiguration

# Load your pre-trained emotion detection CNN model
model = tf.keras.models.load_model('transfer_cnn')  # Replace with the path to your model file

# Define emotion labels (modify as per your model's output)
emotion_labels = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Define the VideoTransformer class for real-time emotion detection
class EmotionDetector(VideoTransformerBase):
    def __init__(self):
        self._prediction = None

    def predict(self, frame):
        # Preprocess the frame for the model
        frame_resized = cv2.resize(frame, (48, 48))
        frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        frame_array = img_to_array(frame_rgb)
        input_tensor = np.expand_dims(frame_array, axis=0)

        # Pass the frame through the model for emotion prediction
        with tf.device('/cpu:0'):  # If using GPU, comment out this line
            output = model.predict(input_tensor)
            predicted_idx = np.argmax(output, axis=1)[0]
            predicted_emotion = emotion_labels[predicted_idx]

        return predicted_emotion

    def transform(self, frame):
        # Predict emotion using the EmotionDetector class
        predicted_emotion = self.predict(frame)

        # Draw bounding box and emotion label on the frame (optional)
        cv2.putText(frame, predicted_emotion, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the processed image (debugging purposes)
        st.image(frame, channels='BGR')

        # Display the predicted emotion
        st.header("Prediction")
        st.subheader("Emotion")
        st.write(predicted_emotion)

        return frame

# Run the Streamlit app
def main():
    st.title('Real-time Emotion Detection')

    # Define the WebRTC configuration to access the webcam
    rtc_configuration = RTCConfiguration(
        iceServers=[{"urls": ["stun:stun.l.google.com:19302"]}],
        mandatory={"OfferToReceiveAudio": True, "OfferToReceiveVideo": True},
    )

    # Start webcam and real-time emotion detection using streamlit-webrtc
    video_transformer = EmotionDetector()
    webrtc_ctx = webrtc_streamer(
        key='emotion-detection',
        rtc_configuration=rtc_configuration,
        video_transformer_factory=lambda: video_transformer,
        async_transform=True,
    )

if __name__ == '__main__':
    main()

Writing app.py


MP4 checker

In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
from PIL import Image
import tempfile

# Load your pre-trained emotion detection CNN model
model = tf.keras.models.load_model('transfer_cnn')  # Replace with the path to your model file

# Define emotion labels (modify as per your model's output)
emotion_labels = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Function to process and classify the frame
def process_frame(frame):
    # Convert the frame to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Preprocess the frame for the model
    frame_resized = cv2.resize(frame_gray, (48, 48))
    frame_array = img_to_array(frame_resized)
    input_tensor = np.expand_dims(frame_array, axis=0)

    # Pass the frame through the model for emotion prediction
    with tf.device('/cpu:0'):  # If using GPU, comment out this line
        output = model.predict(input_tensor)
        predicted_idx = np.argmax(output, axis=1)[0]
        predicted_emotion = emotion_labels[predicted_idx]

    return predicted_emotion

def main():
    st.title('Emotion Detection from MP4 Video')

    # File uploader to upload the MP4 video
    uploaded_file = st.file_uploader("Upload a video (MP4)", type=["mp4"])

    if uploaded_file is not None:
        # Create a temporary file and write the uploaded video content to it
        temp_file = tempfile.NamedTemporaryFile(delete=False)
        temp_file.write(uploaded_file.read())
        temp_file.close()

        # Read the video file using OpenCV
        cap = cv2.VideoCapture(temp_file.name)

        # Perform emotion detection and store emotions and timestamps in lists
        emotions = []
        timestamps = []
        frame_num = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Process the frame
            predicted_emotion = process_frame(frame)
            emotions.append(predicted_emotion)
            timestamps.append(frame_num / cap.get(cv2.CAP_PROP_FPS))
            frame_num += 1

        # Release video capture
        cap.release()

        # Plot emotions over time as a scatterplot
        fig, ax = plt.subplots()
        ax.scatter(timestamps, emotions)
        ax.set_xlabel("Time (seconds)")
        ax.set_ylabel("Emotion")
        st.pyplot(fig)

if __name__ == '__main__':
    main()

Overwriting app.py


Run site

In [ ]:
from pyngrok import ngrok
#Publish Web App (Run this again whenever you make changes)
launch_website()

Click this link to try your web app:


NgrokTunnel: "https://8a6f-35-230-170-157.ngrok-free.app" -> "http://localhost:80"
